    Cargar Datos de Ventas

In [2]:
# Hola

In [3]:
import requests
import pandas as pd
import json

url = 'https://www.sharwinn.com/wp-json/wc/v3/orders'                   # url
params = {
    'consumer_key': 'ck_e88ed88e0a1684cd729a37dee41c2a3cb503b0ec',      # CK
    'consumer_secret': 'cs_0cc67c5057e588a91ba99b38271729baeb83dc56',   # CS
    'per_page': 100,                                                    # número de pedidos por página
    'meta_data': ['_yith_pos_cashier', '[P] _alg_wc_cog_cost']     # Metadata
}
orders = []                                                             # lista para almacenar los pedidos
response = requests.get(url, params=params)                             # Consulta para extraer el número de páginas
npages=round(int(response.headers['X-WP-Total'])/100)+1                 # Conseguimos el número de Páginas

for page in range(1, npages):                                           # Ciclo For para extraer la data
    params['page'] = page
    response = requests.get(url, params=params)
    orders += response.json()

df_agrupada = pd.DataFrame(orders)                                               # Convertimos en DataFrame

df_expanded = df_agrupada.explode('line_items')
df_expanded = pd.concat([df_expanded.drop(['line_items'], axis=1), df_expanded['line_items'].apply(pd.Series)], axis=1)

In [4]:
# Dividimos la Columna yith_pos_data
from pandas.io.json import json_normalize
# Expande los datos de la columna 'yith_pos_data' utilizando json_normalize
df_normalized = json_normalize(df_expanded['yith_pos_data'])
# Concatenar
df_expanded1=df_expanded.copy()
df_normalized1=df_normalized.copy()
df_expanded1 = df_expanded1.reset_index()
df_normalized1 = df_normalized1.reset_index()
df_combined = pd.concat([df_expanded1, df_normalized1], axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_11876\471165115.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_normalized = json_normalize(df_expanded['yith_pos_data'])


In [5]:
df_combined.to_csv('datos.csv', index=False)

    Cargar Datos de productos

In [6]:
import requests
import pandas as pd
import json

url = 'https://www.sharwinn.com/wp-json/wc/v3/products'                   # url
params = {
    'consumer_key': 'ck_e88ed88e0a1684cd729a37dee41c2a3cb503b0ec',      # CK
    'consumer_secret': 'cs_0cc67c5057e588a91ba99b38271729baeb83dc56',   # CS
    'per_page': 100,                                                    # número de pedidos por página
}
products = []                                                             # lista para almacenar los pedidos
response = requests.get(url, params=params)                             # Consulta para extraer el número de páginas
npages=round(int(response.headers['X-WP-Total'])/100)+2                 # Conseguimos el número de Páginas

for page in range(1, npages):                                           # Ciclo For para extraer la data
    params['page'] = page
    response = requests.get(url, params=params)
    products += response.json()

p_agrupada = pd.DataFrame(products)                                               # Convertimos en DataFrame
productos=p_agrupada.copy()

In [7]:
productos.rename(columns={'id': 'product_id'}, inplace=True)

In [8]:
productos.to_csv('productos.csv', index=False)

# EDA Solo Plataformas

In [9]:
import requests
import pandas as pd
import json
data = pd.read_csv('datos.csv')                     # Levantar la tabla guardada
productos = pd.read_csv('productos.csv')                     # Levantar la tabla guardada

In [10]:
# Eliminar columnas inútiles
data.drop(['version',
       'order_key', 'billing', 'shipping',
       'payment_method', 'transaction_id',
       'customer_ip_address', 'customer_user_agent', 'created_via',
       'customer_note', 'date_completed', 'cart_hash', 'number',
       'meta_data', 'tax_lines', 'shipping_lines', 'fee_lines', 'coupon_lines',
       'refunds', 'payment_url', 'is_editable', 'needs_payment',
       'needs_processing', 'date_created_gmt', 'date_modified_gmt',
       'date_completed_gmt', 'date_paid_gmt', 'currency_symbol',
       'multiple_payment_methods', 'pos_payment_details',
       '_links', 'taxes', 'meta_data.1', 'sku', 'price', 'image', 'parent_name',
       'yithPosImage','yith_pos_data'], axis=1, inplace=True)


In [11]:
# Reformatear las fechas
import pandas as pd
from datetime import datetime

# Convertir la columna date_created en un objeto datetime
data['date_created'] = pd.to_datetime(data['date_created'])
data['date_modified'] = pd.to_datetime(data['date_modified'])
data['date_paid'] = pd.to_datetime(data['date_paid'])

In [12]:
# Formatear la columna date_created en YYYY-MM-DD
data['date_created'] = data['date_created'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))
data['date_modified'] = data['date_modified'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))
#data['date_paid'] = data['date_paid'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

In [13]:
# Reformatear las fechas
import pandas as pd
from datetime import datetime

# Convertir la columna date_created en un objeto datetime
productos['date_created'] = pd.to_datetime(productos['date_created'])
productos['date_modified'] = pd.to_datetime(productos['date_modified'])

# Formatear la columna date_created en YYYY-MM-DD
productos['date_created'] = productos['date_created'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))
productos['date_modified'] = productos['date_modified'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

# 'PLATAFORMAS'

In [14]:
data['register_name'] = data['register_name'].fillna('') # Rellenar valores faltantes con cadena vacía
df_mkp = data[data['register_name'].str.contains('Caja 2')]
df_mkp.shape


(3761, 32)

In [15]:
# Concatenar tablas
data_mkp=pd.merge(df_mkp, productos, on='product_id')

### Rotación y última venta

    primera venta

In [16]:
# Primera venta desde su lanzamiento
primer_venta=data_mkp.copy()
# Se agrupa por id_producto y se aplica la función de agregación min() a la columna venta
FV_fecha_venta= primer_venta.groupby('product_id')['date_created_x'].agg(min).reset_index(name='Fecha_Primera_Venta')
print(FV_fecha_venta.shape)
FV_fecha_Creacion= primer_venta.groupby('product_id')['date_created_y'].agg(min).reset_index(name='Fecha_Creacion_producto')
print(FV_fecha_Creacion.shape)
# Unir los dataframe resultantes
primer_venta_fechas=pd.merge(FV_fecha_venta, FV_fecha_Creacion, on='product_id')

(246, 2)
(246, 2)


In [17]:
from datetime import datetime

# Convertir la columna date_created en un objeto datetime
primer_venta_fechas['Fecha_Primera_Venta'] = pd.to_datetime(primer_venta_fechas['Fecha_Primera_Venta'])
primer_venta_fechas['Fecha_Creacion_producto'] = pd.to_datetime(primer_venta_fechas['Fecha_Creacion_producto'])

In [20]:
primer_venta_fechas['diferencia_en_dias'] = (primer_venta_fechas['Fecha_Primera_Venta'] - primer_venta_fechas['Fecha_Creacion_producto']).dt.days
primer_venta_fechas_sorted=primer_venta_fechas.sort_values('diferencia_en_dias', ascending=True)
primer_venta_fechas_sorted.head(10)

,product_id,Fecha_Primera_Venta,Fecha_Creacion_producto,tiempo_hasta_primera_venta,diferencia_en_dias
214,44381,2023-01-16,2023-01-16,0,0
205,44170,2023-01-11,2023-01-11,0,0
169,41666,2022-11-07,2022-11-07,0,0
204,44107,2023-01-10,2023-01-10,0,0
159,41196,2022-10-20,2022-10-20,0,0
203,44094,2023-01-10,2023-01-10,0,0
210,44273,2023-01-14,2023-01-14,0,0
128,38857,2022-06-30,2022-06-30,0,0
180,42837,2022-12-09,2022-12-09,0,0
107,36924,2022-04-27,2022-04-27,0,0


In [21]:
# Eliminar columnas inútiles
primer_venta_fechas_sorted.drop(['Fecha_Primera_Venta', 'Fecha_Creacion_producto'], axis=1, inplace=True)

    última venta

In [22]:
# Primera venta desde su lanzamiento
ultima_venta=data_mkp.copy()
# Se agrupa por id_producto y se aplica la función de agregación min() a la columna venta
UV_fecha_venta= ultima_venta.groupby('product_id')['date_created_x'].agg(min).reset_index(name='Fecha_Primera_Venta')
print(FV_fecha_venta.shape)

FV_fecha_Creacion= ultima_venta.groupby('product_id')['date_created_y'].agg(min).reset_index(name='Fecha_Creacion_producto')
print(FV_fecha_Creacion.shape)
# Unir los dataframe resultantes
primer_venta_fechas=pd.merge(FV_fecha_venta, FV_fecha_Creacion, on='product_id')

(246, 2)
(246, 2)


In [23]:
from datetime import datetime

# Convertir la columna date_created en un objeto datetime
primer_venta_fechas['Fecha_Primera_Venta'] = pd.to_datetime(primer_venta_fechas['Fecha_Primera_Venta'])
primer_venta_fechas['Fecha_Creacion_producto'] = pd.to_datetime(primer_venta_fechas['Fecha_Creacion_producto'])

In [25]:
primer_venta_fechas['diferencia_en_dias'] = (primer_venta_fechas['Fecha_Primera_Venta'] - primer_venta_fechas['Fecha_Creacion_producto']).dt.days
primer_venta_fechas_sorted=primer_venta_fechas.sort_values('diferencia_en_dias', ascending=True)
primer_venta_fechas_sorted.head(10)

,product_id,Fecha_Primera_Venta,Fecha_Creacion_producto,tiempo_hasta_primera_venta,diferencia_en_dias
214,44381,2023-01-16,2023-01-16,0,0
205,44170,2023-01-11,2023-01-11,0,0
169,41666,2022-11-07,2022-11-07,0,0
204,44107,2023-01-10,2023-01-10,0,0
159,41196,2022-10-20,2022-10-20,0,0
203,44094,2023-01-10,2023-01-10,0,0
210,44273,2023-01-14,2023-01-14,0,0
128,38857,2022-06-30,2022-06-30,0,0
180,42837,2022-12-09,2022-12-09,0,0
107,36924,2022-04-27,2022-04-27,0,0


In [26]:
# Eliminar columnas inútiles
primer_venta_fechas_sorted.drop(['Fecha_Primera_Venta', 'Fecha_Creacion_producto'], axis=1, inplace=True)

In [27]:
tiempo_desde_ultima_venta


NameError: name 'tiempo_desde_ultima_venta' is not defined

    Unión

In [ ]:
primer_ultima_venta=pd.merge(primer_venta_fechas_sorted, , on='product_id')

In [ ]:

# Agreamos los nombres de los productos
primer_ultima_venta["nombre_producto"]=data_mkp["name_y"]
# Agreamos la Categoría
primer_ultima_venta["categoria"]=data_mkp["name_y"]
# Agreamos cantidad vendida
primer_ultima_venta["costo"]=data_mkp["name_y"]
# Agreamos stock
primer_ultima_venta["stock"]=data_mkp["name_y"]
# Agreamos Precio
primer_ultima_venta["precio_venta"]=data_mkp["name_y"]
# Agreamos nombre de plataforma
primer_ultima_venta["plataforma"]=data_mkp["name_y"]

    Más vendidos

In [ ]:
sales_by_product = data_mkp.groupby('product_id')['quantity'].sum().reset_index(name='quantity_sold')
sales_by_product_sorted=sales_by_product.sort_values('quantity_sold', ascending=False)
sales_by_product_sorted.head(10)


,product_id,quantity_sold
44,33646,316
108,36942,311
143,40165,292
147,40506,289
55,34060,249
15,31890,231
142,40152,223
130,39018,198
139,39675,175
19,31998,162
